### Setup aus MetEngSim

In [1]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
# import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
# from cobra.io import read_sbml_model, write_sbml_model, load_matlab_model

# loading escher for metabolic network visualization
# import escher
# from escher import Builder
# from time import sleep
# escher.rc['never_ask_before_quit'] = True
# list of available maps
# print(escher.list_available_maps())

# loading Memote, quality assessment of GSMM
# from memote import test_model, snapshot_report

from biolabsim import Host, Strain, Ecol

print('System ready')

System ready


In [2]:
wtHost = Ecol()
wtHost.show_BiotechSetting()
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))
print()
myHost = wtHost.clone_with_mutation('Mut1')
myHost.show_BiotechSetting()
print('Mutation growth rate: {:.2f}'.format(myHost.strain.objective))

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Host: Ecol
Resources: 40
Substrate: None
Strain: WT
Wild type growth rate: 0.87



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.22.1 when using version 0.23.2. This might lead to break

resetting boundaries
Host: Ecol
Resources: 40
Substrate: None
Strain: Mut1
Mutation growth rate: 0.85


### Ändern des Dataframes
#### Hinzufügen des Transkriptionsfaktors

In [10]:
def add_TranscriptionFactor(df):
    newdf = df
    
    # add TF row:
    TF = {
    'RctID':['TF'], 
    'Expression':[2], # noch zu ändern mit Help_PromoterStrength()?
    'Promoter':['GCCCATTGACCATACCGGAAGGAGATAATAAAGTTGCACG'],  # noch zu ändern (echt, random oder Fkt?)
    'ORF': ['ATGGAGATGAAGTAA'], # noch zu ändern (echt, random oder Fkt?)
    'Fluxes': [9], # noch zu ändern mit Help_FluxCalculator()?
    'Expr2Flux': [4.5] # noch zu ändern mit Help_Expr2Flux()?
    }
    dfTF = pd.DataFrame(TF)
    newdf = newdf.append(dfTF, ignore_index=True)
    
    # add TF_regulated and RegType column:
    zeroarray = np.zeros(len(newdf),dtype=int)
    newdf = newdf.assign(TF_regulated=zeroarray) # später zu 0 oder jeweiliger TF-RctID ändern (random oder festgesetzt?)
    newdf = newdf.assign(RegType=zeroarray)
    
    # set Regulation of some Proteins:
    filt = (newdf['RctID']=='TF') # access TF row
    newdf.loc[filt, 'TF_regulated'] = 'TF' # TF regulated by itself
    newdf.loc[filt, 'RegType'] = 'Hill_Activator' # Metabolism Status: TF as activator
    # Future: fct recognizes what substrates are there, then makes TF_act or TF_rep
    filt = (newdf['RctID']=='PFK')
    newdf.loc[filt, 'TF_regulated'] = 'TF'
    newdf.loc[filt, 'RegType'] = 'Hill_Activator'
    filt = (newdf['RctID']=='PGI')
    newdf.loc[filt, 'TF_regulated'] = 'TF' # for now all TF_regulated Proteins regulated by TF
    newdf.loc[filt, 'RegType'] = 'Hill_Activator'
    return newdf

In [11]:
newdf = wtHost.strain.genes_df
newdf = add_TranscriptionFactor(newdf)
newdf

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux,TF_regulated,RegType
0,PFK,0.963,GCCCATTGAAGTGAGTAAGAATACTTTCTAGGTTTGCACG,ATGTTCCGTCGCTAA,7.477382,7.764675,TF,Hill_Activator
1,PFL,3.590,GCCCATTGACAAGATGGCGGGTAAGGTTTAAATTTGCACG,ATGATTTGTATGTAA,0.000000,0.000000,0,0
2,PGI,1.406,GCCCATTGACTCACTCCGCAACCAGGCGTAGTATTGCACG,ATGGATGGGCCATAG,4.860861,3.457227,TF,Hill_Activator
3,PGK,1.093,GCCCATTGAGCCTTTAGCCAAGATGTATTAGGTTTGCACG,ATGGATGGCCAATAA,-16.023526,-14.660134,0,0
4,PGL,4.144,GCCCATTGAAAATTATGGAGCGTCGTATTACACTTGCACG,ATGGGTCAGTTTTGA,4.959985,1.196908,0,0
...,...,...,...,...,...,...,...,...
91,NADTRHD,2.814,GCCCATTGACAGCGCTAACAACCCTTGTTATGCTTGCACG,ATGCGTGTGGAATAA,0.000000,0.000000,0,0
92,NH4t,1.694,GCCCATTGAAAAAGCGTTGGGGTTGGTTTAGCATTGCACG,ATGGATCTGGGCTAA,4.765319,2.813057,0,0
93,O2t,2.133,GCCCATTGAAAGGTGACCGCCGGGATAGTACGATTGCACG,ATGAATACAGAATGA,21.799493,10.220109,0,0
94,PDH,1.005,GCCCATTGATTGTGTGAGGCCGCTAGTTTAGTTTTGCACG,ATGAGTGTCGGCTAA,9.282533,9.236351,0,0


In [5]:
# unnötig but nett to know: random alle Gene durch TF regulieren lassen oder nicht

# import random

# rand_onoff = random.randint(0,1)
    
# newdf['TF_regulated'] = newdf['TF_regulated'].apply(lambda x: x * 0 + onoff)

# newdf

### Regulationsgleichung

In [6]:
# in Help_FluxCalculator:

for Indx in Set_Boundary['lower']:
                myModel.reactions[Indx].lower_bound = FluxEquation(Indx, wtHost.strain.genes_df, myHost.strain.genes_df)
for Indx in Set_Boundary['upper']:
                myModel.reactions[Indx].upper_bound = FluxEquation(Indx, wtHost.strain.genes_df, myHost.strain.genes_df)


# (equationtype = myEquations[RctNewDF.loc[Indx, 'Expr2Flux']])


def FluxEquation(index, WTdf, MTdf): 
    '''gibt abhängig davon, ob das Protein vom TF reguliert wird oder nicht, die Gleichung an, mit der der Flux berechnet wird'''
          
    myTF = WTdf.loc[index, 'TF_regulated']
    
    # linear
    if WTdf[index,'TF_regulated'] == 0: 
        
        Corr_ExprFlux = WTdf.loc[index, 'Expr2Flux'] # WT o. MT
        Flux = MTdf.loc[index, 'NewExpr'] * Corr_ExprFlux 
        return Flux
    
    # Hill
    else:
        
        RegIndx = (WTdf['RctID']==WTdf[index,'TF_regulated'])
        
        MaxExpr = 2 * MTdf.loc[index, 'Expression'].values # bei physiologischem cTF-Wert Flux= mtExpr
        cTF = MTdf.loc[RegIndx, 'Expression'].values # abhängig von geändertem TF-Promotor
        K = WTdf.loc[index, 'Expression'].values # physiologischer Wert von c, Expression/Expr2Flux, beeinflusst Aktivität des TF, WT:nicht durch promotor geändert
        n = 1.95 # Kim, Harold D.; O'Shea, Erin K. (2008): A quantitative model of transcription factor–activated gene expression. DOI: 10.1038/nsmb.1500.  
        
        
        if WTdf[RegIndx,'RegType'] == 'Hill_Activator':

            Flux = MaxExpr / (1+(K/cTF)**n)

            return Flux
    
        elif WTdf[RegIndx,'RegType'] == 'Hill_Repressor':

            Flux = MaxExpr * (1 - 1/ (1+(K/cTF)**n))

            return Flux

NameError: name 'Set_Boundary' is not defined

### Vergleich von WT und MT aus MetEngSim

In [ ]:
wtHost.strain.genes_df

In [ ]:
myHost.strain.genes_df

In [ ]:
mylist = np.array(wtHost.strain.genes_df['Expression'])
mylist2 = np.array(myHost.strain.genes_df['Expression'])
print('Gene expression differences')
np.round(mylist2/mylist,2)

In [ ]:
myl = np.array(wtHost.strain.genes_df['Expr2Flux'])
myl2 = np.array(myHost.strain.genes_df['Expr2Flux'])
# myHost.strain.genes_df
print('Exp2Flux diff')
np.round(myl2/myl,2)

In [ ]:
FluxDiff = myHost.strain.genes_df['Fluxes']/wtHost.strain.genes_df['Fluxes']
print('Flux reaction differences')
np.round(np.nan_to_num(FluxDiff, nan=1, posinf=1, neginf=1),2)